![alt text](https://trello-attachments.s3.amazonaws.com/56b4dc9a5618cc0446578ec6/5c6ffc4673fc7e8ac0a66708/c5f7d8d143f69ffd6683d6ad5b72c58f/Machine-Learning-e-Ciência-de-dados-nas-apostas-esportivas.png)

In [1]:
# Importa Bibliotecas
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
#Setting Pandas to show all columns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format

api_key = '###########################################'

## Usando API e processando dados

In [2]:
requests.get(f'https://apiv2.apifootball.com/?action=get_countries&APIkey={api_key}').json()

[{'country_id': '41', 'country_name': 'England'},
 {'country_id': '46', 'country_name': 'France'}]

In [3]:
requests.get(f'https://apiv2.apifootball.com/?action=get_leagues&country_id=41&APIkey={api_key}').json()

[{'country_id': '41',
  'country_name': 'England',
  'league_id': '149',
  'league_name': 'Championship',
  'league_season': '2019/2020'}]

In [4]:
requests.get(f'https://apiv2.apifootball.com/?action=get_leagues&country_id=46&APIkey={api_key}').json()

[{'country_id': '46',
  'country_name': 'France',
  'league_id': '177',
  'league_name': 'Ligue 2',
  'league_season': '2019/2020'}]

In [5]:
matches = requests.get(f'https://apiv2.apifootball.com/?action=get_events&from=2015-08-01&to=2019-05-05&league_id=149&APIkey={api_key}').json()

In [6]:
matches[0]

{'match_id': '13537',
 'country_id': '41',
 'country_name': 'England',
 'league_id': '149',
 'league_name': 'Championship',
 'match_date': '2017-08-04',
 'match_status': 'Finished',
 'match_time': '20:45',
 'match_hometeam_id': '2634',
 'match_hometeam_name': 'Nottingham',
 'match_hometeam_score': '1',
 'match_awayteam_name': 'Millwall',
 'match_awayteam_id': '2638',
 'match_awayteam_score': '0',
 'match_hometeam_halftime_score': '1',
 'match_awayteam_halftime_score': '0',
 'match_hometeam_extra_score': '',
 'match_awayteam_extra_score': '',
 'match_hometeam_penalty_score': '',
 'match_awayteam_penalty_score': '',
 'match_hometeam_ft_score': '',
 'match_awayteam_ft_score': '',
 'match_hometeam_system': '4 - 1 - 4 - 1',
 'match_awayteam_system': '4 - 4 - 2',
 'match_live': '0',
 'match_round': 'Round 1',
 'match_stadium': '',
 'match_referee': '',
 'goalscorer': [{'time': '41',
   'home_scorer': 'McKay B.',
   'score': '1 - 0',
   'away_scorer': '',
   'info': ''}],
 'cards': [{'time': 

In [7]:
matches[0]['statistics']

[{'type': 'Ball Possession', 'home': '68%', 'away': '32%'},
 {'type': 'Goal Attempts', 'home': '11', 'away': '23'},
 {'type': 'Shots on Goal', 'home': '2', 'away': '6'},
 {'type': 'Shots off Goal', 'home': '5', 'away': '13'},
 {'type': 'Blocked Shots', 'home': '4', 'away': '4'},
 {'type': 'Free Kicks', 'home': '18', 'away': '17'},
 {'type': 'Corner Kicks', 'home': '5', 'away': '4'},
 {'type': 'Offsides', 'home': '4', 'away': '3'},
 {'type': 'Goalkeeper Saves', 'home': '6', 'away': '1'},
 {'type': 'Fouls', 'home': '13', 'away': '15'},
 {'type': 'Yellow Cards', 'home': '0', 'away': '1'}]

In [8]:
match_list = []
for i in range(len(matches)):
    try:
        match_dict = {'date': matches[i]['match_date'], 'home_team': matches[i]['match_hometeam_name'], 'away_team': matches[i]['match_awayteam_name'],
                      'home_score': int(matches[i]['match_hometeam_score']), 'away_score': int(matches[i]['match_awayteam_score'])} 
    except ValueError:
        continue
    for stat in matches[i]['statistics']:
        match_dict['home_' + '_'.join(stat['type'].split(' ')).lower()] = stat['home']
        match_dict['away_' + '_'.join(stat['type'].split(' ')).lower()] = stat['away']
        
    match_list.append(match_dict)

In [9]:
df = pd.DataFrame(match_list)

In [10]:
df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_free_kicks,away_free_kicks,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_distance_covered_(metres),away_distance_covered_(metres),home_attacks,away_attacks,home_dangerous_attacks,away_dangerous_attacks,home_throw-in,away_throw-in,home_goal_kicks,away_goal_kicks,home_pass_success_%,away_pass_success_%,home_total_passes,away_total_passes,home_completed_passes,away_completed_passes,home_tackles,away_tackles
0,2017-08-04,Nottingham,Millwall,1,0,68%,32%,11,23,2,6,5,13,4,4,18,17,5,4,4,3,6,1,13,15,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-04,Sunderland,Derby,1,1,46%,54%,20,11,4,4,11,5,5,2,11,13,6,7,0,1,3,3,12,11,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-05,Aston Villa,Hull City,1,1,51%,49%,12,14,5,5,5,2,2,7,12,14,5,3,3,1,4,4,13,11,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-05,Bristol City,Barnsley,3,1,58%,42%,20,10,11,5,6,4,3,1,12,16,3,2,2,3,4,8,13,9,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-05,Burton,Cardiff,0,1,53%,47%,3,14,0,6,1,2,2,6,17,14,4,6,1,1,5,0,13,15,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 47 columns):
date                              1108 non-null object
home_team                         1108 non-null object
away_team                         1108 non-null object
home_score                        1108 non-null int64
away_score                        1108 non-null int64
home_ball_possession              1009 non-null object
away_ball_possession              1009 non-null object
home_goal_attempts                1009 non-null object
away_goal_attempts                1009 non-null object
home_shots_on_goal                1009 non-null object
away_shots_on_goal                1009 non-null object
home_shots_off_goal               1009 non-null object
away_shots_off_goal               1009 non-null object
home_blocked_shots                1009 non-null object
away_blocked_shots                1009 non-null object
home_free_kicks                   778 non-null object
away_free_kicks 

In [12]:
drop_list = ['home_distance_covered_(metres)', 'away_distance_covered_(metres)', 'home_throw-in',
             'away_throw-in', 'home_goal_kicks', 'away_goal_kicks', 'home_pass_success_%', 'away_pass_success_%',
             'home_total_passes', 'away_total_passes', 'home_completed_passes', 'away_completed_passes', 'home_tackles',
             'away_tackles', 'home_attacks', 'away_attacks', 'home_dangerous_attacks', 'away_dangerous_attacks', 'home_free_kicks',
             'away_free_kicks', 'home_red_cards', 'away_red_cards']
df = df.drop(drop_list, axis=1).reset_index(drop=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 25 columns):
date                     1108 non-null object
home_team                1108 non-null object
away_team                1108 non-null object
home_score               1108 non-null int64
away_score               1108 non-null int64
home_ball_possession     1009 non-null object
away_ball_possession     1009 non-null object
home_goal_attempts       1009 non-null object
away_goal_attempts       1009 non-null object
home_shots_on_goal       1009 non-null object
away_shots_on_goal       1009 non-null object
home_shots_off_goal      1009 non-null object
away_shots_off_goal      1009 non-null object
home_blocked_shots       1009 non-null object
away_blocked_shots       1009 non-null object
home_corner_kicks        1009 non-null object
away_corner_kicks        1009 non-null object
home_offsides            1009 non-null object
away_offsides            1009 non-null object
home_goalkeeper_saves

In [14]:
df['home_yellow_cards'] = df['home_yellow_cards'].fillna(0)
df['away_yellow_cards'] = df['away_yellow_cards'].fillna(0)

In [15]:
df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards
0,2017-08-04,Nottingham,Millwall,1,0,68%,32%,11,23,2,6,5,13,4,4,5,4,4,3,6,1,13,15,0,1
1,2017-08-04,Sunderland,Derby,1,1,46%,54%,20,11,4,4,11,5,5,2,6,7,0,1,3,3,12,11,1,0
2,2017-08-05,Aston Villa,Hull City,1,1,51%,49%,12,14,5,5,5,2,2,7,5,3,3,1,4,4,13,11,2,0
3,2017-08-05,Bristol City,Barnsley,3,1,58%,42%,20,10,11,5,6,4,3,1,3,2,2,3,4,8,13,9,1,2
4,2017-08-05,Burton,Cardiff,0,1,53%,47%,3,14,0,6,1,2,2,6,4,6,1,1,5,0,13,15,1,0


In [16]:
df['home_goals_diff'] = df['home_score'] - df['away_score']
df['away_goals_diff'] = df['away_score'] - df['home_score']

In [17]:
copia = df.copy()

In [18]:
for team in list(copia['home_team'].unique()):
    for i in range(len(copia[copia['home_team'] == team])):
        aux = copia[copia['home_team'] == team].iloc[i:5+i]['home_goals_diff']
        if len(aux) < 5:
            break
        else:
            soma = aux.sum()
            index = list(aux.index)[-1]
            copia.loc[index, 'home_5goals_diff'] = soma

for team in list(copia['away_team'].unique()):
    for i in range(len(copia[copia['away_team'] == team])):
        aux = copia[copia['away_team'] == team].iloc[i:5+i]['away_goals_diff']
        if len(aux) < 5:
            break
        else:
            soma = aux.sum()
            index = list(aux.index)[-1]
            copia.loc[index, 'away_5goals_diff'] = soma

In [19]:
for i in range(len(copia[copia['home_team'] == 'Nottingham'])):
    if len(copia[copia['home_team'] == 'Nottingham'].iloc[i:5+i,:]) < 5:
        break
    else:
        print(copia[copia['home_team'] == 'Nottingham'].iloc[i:5+i,:][['home_goals_diff', 'home_5goals_diff']])

     home_goals_diff  home_5goals_diff
0                  1               nan
43                 1               nan
50                -2               nan
92                -1               nan
115               -2             -3.00
     home_goals_diff  home_5goals_diff
43                 1               nan
50                -2               nan
92                -1               nan
115               -2             -3.00
127                1             -3.00
     home_goals_diff  home_5goals_diff
50                -2               nan
92                -1               nan
115               -2             -3.00
127                1             -3.00
150                2             -2.00
     home_goals_diff  home_5goals_diff
92                -1               nan
115               -2             -3.00
127                1             -3.00
150                2             -2.00
186                4              4.00
     home_goals_diff  home_5goals_diff
115               -2     

In [20]:
copia[copia['away_team'] == 'Nottingham'].tail()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff
1002,2019-03-16,Ipswich,Nottingham,1,1,49%,51%,16,12,6,5,7,2,3,5,6,5,4,2,4,5,16,12,0,2,0,0,-2.00,-4.00
1031,2019-04-06,Rotherham,Nottingham,2,1,47%,53%,17,13,5,4,7,4,5,5,4,4,1,4,3,3,10,14,0,1,1,-1,1.00,-3.00
1037,2019-04-09,Sheffield Wed,Nottingham,3,0,54%,46%,14,14,5,6,5,4,4,4,1,2,1,2,6,2,14,15,2,2,3,-3,5.00,-6.00
1059,2019-04-19,Sheffield Utd,Nottingham,2,0,63%,37%,14,9,4,1,8,4,2,4,9,2,1,1,1,2,16,8,3,2,2,-2,5.00,-8.00
1088,2019-04-27,QPR,Nottingham,0,1,55%,45%,21,7,7,4,12,2,2,1,8,4,0,4,3,7,11,15,1,2,-1,1,0.00,-5.00


In [22]:
copia.reset_index(inplace=True, drop=True)

In [23]:
copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 29 columns):
date                     1108 non-null object
home_team                1108 non-null object
away_team                1108 non-null object
home_score               1108 non-null int64
away_score               1108 non-null int64
home_ball_possession     1009 non-null object
away_ball_possession     1009 non-null object
home_goal_attempts       1009 non-null object
away_goal_attempts       1009 non-null object
home_shots_on_goal       1009 non-null object
away_shots_on_goal       1009 non-null object
home_shots_off_goal      1009 non-null object
away_shots_off_goal      1009 non-null object
home_blocked_shots       1009 non-null object
away_blocked_shots       1009 non-null object
home_corner_kicks        1009 non-null object
away_corner_kicks        1009 non-null object
home_offsides            1009 non-null object
away_offsides            1009 non-null object
home_goalkeeper_saves

In [24]:
copia.dropna(inplace=True)

In [25]:
copia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 94 to 1107
Data columns (total 29 columns):
date                     871 non-null object
home_team                871 non-null object
away_team                871 non-null object
home_score               871 non-null int64
away_score               871 non-null int64
home_ball_possession     871 non-null object
away_ball_possession     871 non-null object
home_goal_attempts       871 non-null object
away_goal_attempts       871 non-null object
home_shots_on_goal       871 non-null object
away_shots_on_goal       871 non-null object
home_shots_off_goal      871 non-null object
away_shots_off_goal      871 non-null object
home_blocked_shots       871 non-null object
away_blocked_shots       871 non-null object
home_corner_kicks        871 non-null object
away_corner_kicks        871 non-null object
home_offsides            871 non-null object
away_offsides            871 non-null object
home_goalkeeper_saves    871 non-null ob

In [26]:
copia.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff
94,2017-09-23,Aston Villa,Nottingham,2,1,36%,64%,11,13,4,5,2,4,5,4,3,4,3,1,4,2,14,9,1,3,1,-1,3.00,-2.00
95,2017-09-23,Bolton,Brentford,0,3,43%,57%,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00
97,2017-09-23,Fulham,Middlesbrough,1,1,62%,38%,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00
99,2017-09-23,Norwich,Bristol City,0,0,49%,51%,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00
100,2017-09-23,Preston,Millwall,0,0,62%,38%,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00


In [27]:
final_df = copia.copy()

In [28]:
final_df['home_ball_possession'] = final_df['home_ball_possession'].apply(lambda x: int(x.split('%')[0]))
final_df['away_ball_possession'] = final_df['away_ball_possession'].apply(lambda x: int(x.split('%')[0]))

In [29]:
columns_list = ['home_goal_attempts', 'away_goal_attempts', 'home_shots_on_goal', 
                'away_shots_on_goal', 'home_shots_off_goal', 'away_shots_off_goal', 
                'home_blocked_shots', 'away_blocked_shots', 'home_corner_kicks', 
                'away_corner_kicks', 'home_offsides', 'away_offsides', 'home_goalkeeper_saves',
                'away_goalkeeper_saves', 'home_fouls', 'away_fouls','home_yellow_cards', 'away_yellow_cards']
for column in columns_list:
    final_df[column] = final_df[column].astype('int64')

In [30]:
final_df.reset_index(inplace=True, drop=True)

In [31]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 29 columns):
date                     871 non-null object
home_team                871 non-null object
away_team                871 non-null object
home_score               871 non-null int64
away_score               871 non-null int64
home_ball_possession     871 non-null int64
away_ball_possession     871 non-null int64
home_goal_attempts       871 non-null int64
away_goal_attempts       871 non-null int64
home_shots_on_goal       871 non-null int64
away_shots_on_goal       871 non-null int64
home_shots_off_goal      871 non-null int64
away_shots_off_goal      871 non-null int64
home_blocked_shots       871 non-null int64
away_blocked_shots       871 non-null int64
home_corner_kicks        871 non-null int64
away_corner_kicks        871 non-null int64
home_offsides            871 non-null int64
away_offsides            871 non-null int64
home_goalkeeper_saves    871 non-null int64
away_goalkee

In [32]:
final_df.describe()

,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff
count,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00,871.00
mean,1.47,1.18,51.51,48.49,13.70,11.30,4.45,3.74,5.57,4.54,3.69,3.03,5.66,4.72,1.61,1.72,2.55,2.98,11.76,12.41,1.64,1.95,0.29,-0.29,1.46,-1.52
std,1.18,1.07,10.42,10.42,4.74,4.31,2.35,2.14,2.57,2.38,2.30,2.02,2.75,2.56,1.40,1.48,1.75,1.98,3.53,3.66,1.22,1.33,1.60,1.60,4.21,4.09
min,0.00,0.00,8.00,22.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,1.00,0.00,0.00,-5.00,-6.00,-13.00,-13.00
25%,1.00,0.00,44.00,41.00,11.00,8.00,3.00,2.00,4.00,3.00,2.00,2.00,4.00,3.00,1.00,1.00,1.00,2.00,9.00,10.00,1.00,1.00,-1.00,-1.00,-1.00,-4.00
50%,1.00,1.00,52.00,48.00,13.00,11.00,4.00,3.00,5.00,4.00,3.00,3.00,5.00,4.00,1.00,1.00,2.00,3.00,12.00,12.00,2.00,2.00,0.00,0.00,1.00,-2.00
75%,2.00,2.00,59.00,56.00,16.00,14.00,6.00,5.00,7.00,6.00,5.00,4.00,7.00,6.00,2.00,3.00,4.00,4.00,14.00,15.00,2.00,3.00,1.00,1.00,4.00,1.00
max,6.00,5.00,78.00,92.00,36.00,28.00,15.00,12.00,16.00,15.00,13.00,12.00,15.00,16.00,7.00,9.00,11.00,12.00,23.00,25.00,6.00,7.00,6.00,5.00,15.00,10.00


In [33]:
final_df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff
0,2017-09-23,Aston Villa,Nottingham,2,1,36,64,11,13,4,5,2,4,5,4,3,4,3,1,4,2,14,9,1,3,1,-1,3.00,-2.00
1,2017-09-23,Bolton,Brentford,0,3,43,57,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00
2,2017-09-23,Fulham,Middlesbrough,1,1,62,38,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00
3,2017-09-23,Norwich,Bristol City,0,0,49,51,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00
4,2017-09-23,Preston,Millwall,0,0,62,38,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00


In [34]:
def result(df):
    if df['home_score'] > df['away_score']:
        return 'H'
    elif df['home_score'] < df['away_score']:
        return 'A'
    else:
        return 'D'

In [35]:
final_df['result'] = final_df.apply(result, axis=1)

In [36]:
final_df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff,result
0,2017-09-23,Aston Villa,Nottingham,2,1,36,64,11,13,4,5,2,4,5,4,3,4,3,1,4,2,14,9,1,3,1,-1,3.00,-2.00,H
1,2017-09-23,Bolton,Brentford,0,3,43,57,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00,A
2,2017-09-23,Fulham,Middlesbrough,1,1,62,38,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00,D
3,2017-09-23,Norwich,Bristol City,0,0,49,51,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00,D
4,2017-09-23,Preston,Millwall,0,0,62,38,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00,D


In [37]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 30 columns):
date                     871 non-null object
home_team                871 non-null object
away_team                871 non-null object
home_score               871 non-null int64
away_score               871 non-null int64
home_ball_possession     871 non-null int64
away_ball_possession     871 non-null int64
home_goal_attempts       871 non-null int64
away_goal_attempts       871 non-null int64
home_shots_on_goal       871 non-null int64
away_shots_on_goal       871 non-null int64
home_shots_off_goal      871 non-null int64
away_shots_off_goal      871 non-null int64
home_blocked_shots       871 non-null int64
away_blocked_shots       871 non-null int64
home_corner_kicks        871 non-null int64
away_corner_kicks        871 non-null int64
home_offsides            871 non-null int64
away_offsides            871 non-null int64
home_goalkeeper_saves    871 non-null int64
away_goalkee

In [38]:
final_df['home_goals_shots'] = final_df['home_score'] / final_df['home_shots_on_goal']
final_df['away_goals_shots'] = final_df['away_score'] / final_df['away_shots_on_goal']
final_df['home_goals_shots'].replace(np.inf, df['home_score'], inplace=True)
final_df['away_goals_shots'].replace(np.inf, df['away_score'], inplace=True)
final_df['home_goals_shots'].replace(np.nan, 0, inplace=True)
final_df['away_goals_shots'].replace(np.nan, 0, inplace=True)

In [40]:
final_df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff,result,home_goals_shots,away_goals_shots
0,2017-09-23,Aston Villa,Nottingham,2,1,36,64,11,13,4,5,2,4,5,4,3,4,3,1,4,2,14,9,1,3,1,-1,3.00,-2.00,H,0.50,0.20
1,2017-09-23,Bolton,Brentford,0,3,43,57,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00,A,0.00,0.30
2,2017-09-23,Fulham,Middlesbrough,1,1,62,38,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00,D,0.25,0.33
3,2017-09-23,Norwich,Bristol City,0,0,49,51,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00,D,0.00,0.00
4,2017-09-23,Preston,Millwall,0,0,62,38,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00,D,0.00,0.00


## Adicionando o Elo Score

In [41]:
from io import StringIO
list_df = []
for data in final_df.date.unique():
    elo_csv = requests.get(f"http://api.clubelo.com/{data}", allow_redirects=True)
    elo_csv = str(elo_csv.content,'utf-8')
    elo_csv = StringIO(elo_csv) 
    elo_df = pd.read_csv(elo_csv)
    list_df.append(elo_df)

In [42]:
elo_df_final = pd.concat(list_df)
elo_df_final.shape

(109027, 7)

In [43]:
elo_df_final.head()

,Rank,Club,Country,Level,Elo,From,To
0,1,Real Madrid,ESP,1,"2,062.51",2017-09-21,2017-09-23
1,2,Barcelona,ESP,1,"2,046.01",2017-09-20,2017-09-23
2,3,Bayern,GER,1,"1,967.59",2017-09-23,2017-09-26
3,4,Atletico,ESP,1,"1,960.64",2017-09-21,2017-09-23
4,5,Juventus,ITA,1,"1,956.76",2017-09-21,2017-09-23


In [44]:
elo_df_final.drop_duplicates(inplace=True)
elo_df_final.reset_index(drop=True)

,Rank,Club,Country,Level,Elo,From,To
0,1,Real Madrid,ESP,1,"2,062.51",2017-09-21,2017-09-23
1,2,Barcelona,ESP,1,"2,046.01",2017-09-20,2017-09-23
2,3,Bayern,GER,1,"1,967.59",2017-09-23,2017-09-26
3,4,Atletico,ESP,1,"1,960.64",2017-09-21,2017-09-23
4,5,Juventus,ITA,1,"1,956.76",2017-09-21,2017-09-23
...,...,...,...,...,...,...,...
44273,None,Luch Minsk,BLR,1,"1,152.11",2019-05-05,2019-05-11
44274,None,Vitosha Bistritsa,BUL,1,"1,149.11",2019-05-03,2019-05-08
44275,None,Torpedo Minsk,BLR,1,"1,133.82",2019-05-05,2019-05-12
44276,None,Dundee,SCO,1,"1,123.46",2019-05-05,2019-05-11


In [45]:
elo_df_eng = elo_df_final[elo_df_final['Country'] == 'ENG'].reset_index(drop=True)

In [46]:
elo_df_eng.head()

,Rank,Club,Country,Level,Elo,From,To
0,6,Chelsea,ENG,1,"1,915.90",2017-09-18,2017-09-23
1,7,Man City,ENG,1,"1,906.20",2017-09-17,2017-09-23
2,10,Tottenham,ENG,1,"1,892.49",2017-09-21,2017-09-23
3,11,Man United,ENG,1,"1,888.46",2017-09-21,2017-09-23
4,14,Arsenal,ENG,1,"1,845.44",2017-09-18,2017-09-23


In [47]:
copy_final_df = final_df.copy()
copy_final_df = copy_final_df.reset_index(drop=True)

In [48]:
copy_final_df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff,result,home_goals_shots,away_goals_shots
0,2017-09-23,Aston Villa,Nottingham,2,1,36,64,11,13,4,5,2,4,5,4,3,4,3,1,4,2,14,9,1,3,1,-1,3.00,-2.00,H,0.50,0.20
1,2017-09-23,Bolton,Brentford,0,3,43,57,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00,A,0.00,0.30
2,2017-09-23,Fulham,Middlesbrough,1,1,62,38,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00,D,0.25,0.33
3,2017-09-23,Norwich,Bristol City,0,0,49,51,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00,D,0.00,0.00
4,2017-09-23,Preston,Millwall,0,0,62,38,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00,D,0.00,0.00


In [49]:
copy_final_df.home_team.unique()

array(['Aston Villa', 'Bolton', 'Fulham', 'Norwich', 'Preston', 'QPR',
       'Sunderland', 'Sheffield Wed', 'Barnsley', 'Brentford',
       'Bristol City', 'Burton', 'Cardiff', 'Hull City', 'Ipswich',
       'Middlesbrough', 'Millwall', 'Nottingham', 'Birmingham',
       'Sheffield Utd', 'Reading', 'Wolves', 'Leeds', 'Derby', 'Wigan',
       'Stoke', 'West Brom', 'Blackburn', 'Swansea', 'Rotherham', 'Hull'],
      dtype=object)

In [50]:
elo_df_eng.Club.unique()

array(['Chelsea', 'Man City', 'Tottenham', 'Man United', 'Arsenal',
       'Liverpool', 'Everton', 'Leicester', 'Southampton', 'Stoke',
       'West Ham', 'Burnley', 'Swansea', 'West Brom', 'Newcastle',
       'Bournemouth', 'Watford', 'Crystal Palace', 'Brighton',
       'Middlesbrough', 'Leeds', 'Sheffield Weds', 'Fulham', 'Hull',
       'Sunderland', 'Huddersfield', 'Cardiff', 'Reading', 'Norwich',
       'Preston', 'Wolves', 'Derby', 'Aston Villa', 'Brentford',
       'Ipswich', 'Bristol City', 'QPR', 'Forest', 'Barnsley', 'Burton',
       'Sheffield United', 'Millwall', 'Birmingham', 'Bolton',
       'Rotherham', 'Wigan', 'Blackburn'], dtype=object)

In [51]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
for team in list(copy_final_df.home_team.unique()):
    print(team)
    print(process.extract(team, elo_df_eng.Club.unique(), limit=2), end='')
    print('\n')

Aston Villa
[('Aston Villa', 100), ('Stoke', 54)]

Bolton
[('Bolton', 100), ('Burton', 67)]

Fulham
[('Fulham', 100), ('Southampton', 60)]

Norwich
[('Norwich', 100), ('Ipswich', 57)]

Preston
[('Preston', 100), ('Forest', 62)]

QPR
[('QPR', 100), ('Preston', 60)]

Sunderland
[('Sunderland', 100), ('Huddersfield', 55)]

Sheffield Wed
[('Sheffield Weds', 96), ('Sheffield United', 83)]

Barnsley
[('Barnsley', 100), ('Burnley', 80)]

Brentford
[('Brentford', 100), ('Watford', 62)]

Bristol City
[('Bristol City', 100), ('Man City', 86)]

Burton
[('Burton', 100), ('Brighton', 71)]

Cardiff
[('Cardiff', 100), ('Watford', 43)]

Hull City
[('Hull', 90), ('Man City', 64)]

Ipswich
[('Ipswich', 100), ('Norwich', 57)]

Middlesbrough
[('Middlesbrough', 100), ('Leeds', 54)]

Millwall
[('Millwall', 100), ('Hull', 45)]

Nottingham
[('Tottenham', 74), ('Birmingham', 60)]

Birmingham
[('Birmingham', 100), ('Brighton', 56)]

Sheffield Utd
[('Sheffield United', 90), ('Sheffield Weds', 81)]

Reading
[('Re

In [52]:
copy_final_df = copy_final_df[(copy_final_df['home_team'] != 'Nottingham') & (copy_final_df['away_team'] != 'Nottingham')]

In [53]:
for key, row in copy_final_df.iterrows():
    choices = list(elo_df_eng.Club.unique())
    home_team = process.extract(row['home_team'], choices, limit=2)[0][0]
    home_team_elo = elo_df_eng[(elo_df_eng['Club'] == home_team) & ((elo_df_eng['To'] == row['date']) | (elo_df_eng['From'] == row['date']))]['Elo']
    away_team = process.extract(row['away_team'], choices, limit=2)[0][0]
    away_team_elo = elo_df_eng[(elo_df_eng['Club'] == away_team) & ((elo_df_eng['To'] == row['date']) | (elo_df_eng['From'] == row['date']))]['Elo']
    if not home_team_elo.empty:
        copy_final_df.loc[key,'home_elo'] = home_team_elo.values[0]
    if not away_team_elo.empty:
        copy_final_df.loc[key,'away_elo'] = away_team_elo.values[0]

In [54]:
copy_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 1 to 870
Data columns (total 34 columns):
date                     798 non-null object
home_team                798 non-null object
away_team                798 non-null object
home_score               798 non-null int64
away_score               798 non-null int64
home_ball_possession     798 non-null int64
away_ball_possession     798 non-null int64
home_goal_attempts       798 non-null int64
away_goal_attempts       798 non-null int64
home_shots_on_goal       798 non-null int64
away_shots_on_goal       798 non-null int64
home_shots_off_goal      798 non-null int64
away_shots_off_goal      798 non-null int64
home_blocked_shots       798 non-null int64
away_blocked_shots       798 non-null int64
home_corner_kicks        798 non-null int64
away_corner_kicks        798 non-null int64
home_offsides            798 non-null int64
away_offsides            798 non-null int64
home_goalkeeper_saves    798 non-null int64
away_goalkee

In [55]:
copy_final_df.head()

,date,home_team,away_team,home_score,away_score,home_ball_possession,away_ball_possession,home_goal_attempts,away_goal_attempts,home_shots_on_goal,away_shots_on_goal,home_shots_off_goal,away_shots_off_goal,home_blocked_shots,away_blocked_shots,home_corner_kicks,away_corner_kicks,home_offsides,away_offsides,home_goalkeeper_saves,away_goalkeeper_saves,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_goals_diff,away_goals_diff,home_5goals_diff,away_5goals_diff,result,home_goals_shots,away_goals_shots,home_elo,away_elo
1,2017-09-23,Bolton,Brentford,0,3,43,57,10,22,3,10,5,6,2,6,5,6,1,3,7,3,22,13,2,1,-3,3,-9.00,-1.00,A,0.00,0.30,"1,324.42","1,464.26"
2,2017-09-23,Fulham,Middlesbrough,1,1,62,38,10,14,4,3,5,8,1,3,6,3,2,1,2,3,16,8,2,1,0,0,0.00,1.00,D,0.25,0.33,"1,543.04","1,563.94"
3,2017-09-23,Norwich,Bristol City,0,0,49,51,15,10,1,3,10,5,4,2,3,6,2,3,3,1,15,19,1,1,0,0,1.00,0.00,D,0.00,0.00,"1,509.35","1,463.22"
4,2017-09-23,Preston,Millwall,0,0,62,38,12,10,2,1,7,5,3,4,7,7,1,1,1,2,18,16,3,1,0,0,5.00,-2.00,D,0.00,0.00,"1,502.02","1,380.96"
5,2017-09-23,QPR,Burton,0,0,63,37,13,5,4,1,3,2,6,2,8,6,1,2,1,4,17,11,0,0,0,0,4.00,-10.00,D,0.00,0.00,"1,447.36","1,405.12"


In [56]:
copy_final_df.to_csv('class_data.csv', index=False, sep=';')